# Decision Tree
Aug 5, 2021 

## Load the data

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("../../S25/golf_data.csv")
print(data.shape)
data

(14, 5)


,Outlook,Temperature,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


In [4]:
X = data.iloc[:, :-1]
X

,Outlook,Temperature,Humidity,Windy
0,Rainy,Hot,High,False
1,Rainy,Hot,High,True
2,Overcast,Hot,High,False
3,Sunny,Mild,High,False
4,Sunny,Cool,Normal,False
5,Sunny,Cool,Normal,True
6,Overcast,Cool,Normal,True
7,Rainy,Mild,High,False
8,Rainy,Cool,Normal,False
9,Sunny,Mild,Normal,False


In [5]:
y = data["Play Golf"]
y

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Play Golf, dtype: object

In [6]:
class Node:

    def __init__(self):
        self.feature = None
        self.children = {}
        self.X = None
        self.y = None
        self.leaf = False

    def predict(self):
        uni, count = np.unique(self.y.values, return_counts=True)

        p = count/self.y.shape[0]
        ix = np.argmax(count)
        d = {k:v for k,v in zip(uni,p)}
        return d

In [24]:
class DecisionTree:
    def __init__(self):
        self.root = None

    def entropy(self, y):
        uni, count = np.unique(y, return_counts=True)
        p = count/y.shape[0] # probability of each category
        ent = -( p*np.log2(p) ).sum()
        return ent
    
    def feature_entropy(self, feature, X, y):
        uni, count = np.unique(X[feature], return_counts=True)
        w = count/y.shape[0]
        ent = 0
        for i,val in enumerate(uni):
            ent += w[i]*self.entropy(y[ X[feature] == val ])
        return ent

    def split_data(self, feature, X, y):
        print("+"*30,f"split_data called on {feature}", "+"*30)
        children = {}
        unique = np.unique(X[feature])
        for val in unique:
            child_node = Node() # create a node at all unique values of feature
            child_node.feature = val
            child_node.X = X[ X[feature]==val ].drop([feature], axis=1)
            child_node.y = y[ X[feature]==val ]
            children[val] = child_node # inserting a new key in children dict
            print(val, child_node.y.values)
        print(f"children of {feature}: {children.keys()}")
        print()
        return children

    
    def make_tree(self, node, X, y):
        print("="*15, f" make_tree  is called on {node.feature}", "="*15)
        ent_before_split = self.entropy(y)
        features = X.columns

        # compute info gain for every feature
        info_gain = []
        for feature in features:
            ig = ent_before_split - self.feature_entropy(feature, X, y)
            info_gain.append(ig)

        # best feature
        ix = np.argmax(info_gain)
        
        best_feature = features[ix]
        d = {f:i_gain for f,i_gain in zip(features, info_gain)}
        print("Info gains of features:", d)
        print("best feature: ", best_feature)

        if info_gain[ix] > 0:
            node.feature = best_feature
            # node.X = X
            # node.y = y
            # split data across unique value of the best feature
            node.children = self.split_data(best_feature, X, y)
            for child in node.children.values():
                self.make_tree(child, child.X, child.y)
            return 
        else:
            node.leaf = True
            node.X = X
            node.y = y

            print("leaf node", node.y.values)
            return 

    def fit(self, X, y):
        self.root = Node()
        self.make_tree(self.root, X, y)

    def predict_recursive(self, node, X):
        if node.leaf:
            return node.predict()
        
        val = X[node.feature]
        return self.predict_recursive(node.children[val], X)

    def predict(self, X):
        return self.predict_recursive(self.root, X)

    def print_tree_recursive(self, node, indent):
        if node.leaf:
            # print(indent+str(node.feature), "-->", node.predict())
            print(node.predict())
            return
        print(indent+str(node.feature), "-->")
        for f_val, child in node.children.items():
            print(indent+str(f_val), "-->", end=" ")
            self.print_tree_recursive(child, indent+"\t")
            
    def print_tree(self):
        self.print_tree_recursive(self.root, indent="")


In [8]:
X_train = X.iloc[:10, :]
y_train = y[:10]
print(X_train.shape, y_train.shape)
# dt.fit()

(10, 4) (10,)


In [9]:
# y_train
X_train

,Outlook,Temperature,Humidity,Windy
0,Rainy,Hot,High,False
1,Rainy,Hot,High,True
2,Overcast,Hot,High,False
3,Sunny,Mild,High,False
4,Sunny,Cool,Normal,False
5,Sunny,Cool,Normal,True
6,Overcast,Cool,Normal,True
7,Rainy,Mild,High,False
8,Rainy,Cool,Normal,False
9,Sunny,Mild,Normal,False


In [25]:
# train the dt
dt = DecisionTree()
dt.fit(X_train, y_train)
dt.print_tree()

===============  make_tree  is called on None ===============
Info gains of features: {'Outlook': 0.3219280948873623, 'Temperature': 0.09546184423832171, 'Humidity': 0.12451124978365313, 'Windy': 0.0912774462416801}
best feature:  Outlook
++++++++++++++++++++++++++++++ split_data called on Outlook ++++++++++++++++++++++++++++++
Overcast ['Yes' 'Yes']
Rainy ['No' 'No' 'No' 'Yes']
Sunny ['Yes' 'Yes' 'No' 'Yes']
children of Outlook: dict_keys(['Overcast', 'Rainy', 'Sunny'])

===============  make_tree  is called on Overcast ===============
Info gains of features: {'Temperature': -0.0, 'Humidity': -0.0, 'Windy': -0.0}
best feature:  Temperature
leaf node ['Yes' 'Yes']
===============  make_tree  is called on Rainy ===============
Info gains of features: {'Temperature': 0.8112781244591328, 'Humidity': 0.8112781244591328, 'Windy': 0.12255624891826566}
best feature:  Temperature
++++++++++++++++++++++++++++++ split_data called on Temperature ++++++++++++++++++++++++++++++
Cool ['Yes']
Hot ['N

In [26]:
# train the dt
dt = DecisionTree()
dt.fit(X, y)
dt.print_tree()

===============  make_tree  is called on None ===============
Info gains of features: {'Outlook': 0.24674981977443933, 'Temperature': 0.02922256565895487, 'Humidity': 0.15183550136234159, 'Windy': 0.04812703040826949}
best feature:  Outlook
++++++++++++++++++++++++++++++ split_data called on Outlook ++++++++++++++++++++++++++++++
Overcast ['Yes' 'Yes' 'Yes' 'Yes']
Rainy ['No' 'No' 'No' 'Yes' 'Yes']
Sunny ['Yes' 'Yes' 'No' 'Yes' 'No']
children of Outlook: dict_keys(['Overcast', 'Rainy', 'Sunny'])

===============  make_tree  is called on Overcast ===============
Info gains of features: {'Temperature': -0.0, 'Humidity': -0.0, 'Windy': -0.0}
best feature:  Temperature
leaf node ['Yes' 'Yes' 'Yes' 'Yes']
===============  make_tree  is called on Rainy ===============
Info gains of features: {'Temperature': 0.5709505944546686, 'Humidity': 0.9709505944546686, 'Windy': 0.01997309402197489}
best feature:  Humidity
++++++++++++++++++++++++++++++ split_data called on Humidity ++++++++++++++++++++